In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define the function for ALMA
def alma(data, window_size:int, sigma:float, offset:float):
    m = offset * (window_size - 1)
    s = window_size / sigma
    w = np.exp(-(np.arange(window_size) - m)**2 / (2 * s**2))
    w /= w.sum()
    return np.convolve(data, w, mode='valid')

In [ ]:
# Define the function for the optimal ALMA, which takes the fast and slow ALMA as parameters
# Uses kalman filter to determine the optimal ALMA
def optimal_alma(fast_alma, slow_alma):
    # Change the size of the fast alma to slow alma
    fast_alma = fast_alma[:len(slow_alma)]
    
    # Initialize the optimal ALMA
    optimal_alma = []
    # Initialize the kalman filter
    kalman_gain = 0.5
    # Iterate through the fast and slow ALMA
    for i in range(len(fast_alma)):
        # Compute for the optimal ALMA
        optimal_alma.append(kalman_gain * fast_alma[i] + (1 - kalman_gain) * slow_alma[i])
    return optimal_alma

In [ ]:
# Load data
closing_prices = np.genfromtxt('./data_test/PSEI.csv', delimiter=',', skip_header=1, usecols=4)
closing_prices

In [ ]:
# Function for MEAN ABSOLUTE PERCENTAGE ERROR (MAPE)
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# Define a range of window_sizes, sigma, and offset
slow_ws = np.arange(10,20, 1)
slow_sigma = np.arange(-20, 20, 1)
slow_offset = np.arange(0.85, 1.15, 0.1)

fast_ws = np.arange(1,10, 1)
fast_sigma = np.arange(-20, 20, 1)
fast_offset = np.arange(0.85, 1.15, 0.1)

In [ ]:
# Test each combination of window_size, sigma, and offset and find the best combination that yields the lowest MAPE

# Initialize a dictionary to store the MAPE and the corresponding window_size, sigma, and offset
dict_mape = {}


# Iterate through the slow window_size
for slow_ws_ in slow_ws:
    # Iterate through the slow sigma
    for slow_sigma_ in slow_sigma:
        # Iterate through the slow offset
        for slow_offset_ in slow_offset:
            # Iterate through the fast window_size
            for fast_ws_ in fast_ws:
                # Iterate through the fast sigma
                for fast_sigma_ in fast_sigma:
                    # Iterate through the fast offset
                    for fast_offset_ in fast_offset:
                        # Compute for the slow and fast ALMA
                        slow_alma = alma(closing_prices, slow_ws_, slow_sigma_, slow_offset_)
                        fast_alma = alma(closing_prices, fast_ws_, fast_sigma_, fast_offset_)
                        # Compute for the optimal ALMA
                        optimal_alma_ = optimal_alma(fast_alma, slow_alma)
                        # Compute for the MAPE (based on the last 1000 data points)
                        mape_ = mape(closing_prices[-1000:], optimal_alma_[-1000:])
                        # Store the MAPE and the corresponding window_size, sigma, and offset
                        dict_mape[mape_] = [slow_ws_, slow_sigma_, slow_offset_, fast_ws_, fast_sigma_, fast_offset_]

                        # Print the MAPE
                        print(f'Current MAPE: {mape_}', end='\r')

In [ ]:
# Find the lowest MAPE
min_mape = min(dict_mape.keys())
# Print the parameters that yields the lowest MAPE
print(f'Lowest MAPE: {min_mape}\nSlow ALMA: {dict_mape[min_mape][0]}, {dict_mape[min_mape][1]}, {dict_mape[min_mape][2]}\nFast ALMA: {dict_mape[min_mape][3]}, {dict_mape[min_mape][4]}, {dict_mape[min_mape][5]}')